In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, Adadelta, Adam, RMSprop, Adagrad, Nadam, Adamax

data = pd.read_csv('data/winequality-red.csv')
y = data['quality']
X = data.drop(['quality'], axis=1)

In [8]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)
X_train, X_val, y_tarin, y_val = train_test_split(X, y, test_size=0.2, random_state=2017)

In [9]:
# 定义一个创建模型的函数
def create_model(opt):
    model = Sequential()
    model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model


In [10]:
# 定义一个训练期间使用的回调函数
def create_callbacks(opt):
    callbacks = [
        EarlyStopping(monitor='val_acc', patience=200, verbose=2),
        ModelCheckpoint('best_model_'+opt+'.h5', monitor='val_acc', save_best_only=True, verbose=0)
        
    ]
    return callbacks


In [11]:
opts = dict({
    'sgd': SGD(),
    'sgd-0001': SGD(lr=0.0001, decay=0.00001),
    'adam': Adam(),
    'adadelta': Adadelta(),
    'rmsprop': RMSprop(),
    'rmsprop-0001': RMSprop(lr=0.0001),
    'nadam': Nadam(),
    'adamax': Adamax(),
}
)

In [12]:
# 训练网络并存储结果
results = []
for opt in opts:
    model = create_model(opt)
    callbacks = create_callbacks(opt)
    model.compile(loss='mse', optimizer=opts[opt], metrics=['accuracy'])
    hist = model.fit(X_train.values, y_train, batch_size=128, epochs=5000, validation_data=(X_val.values, y_val), verbose=0, callbacks=callbacks)
    best_epoch = np.argmax(hist.history['val_acc'])
    best_acc = hist.history['val_acc'][best_epoch]
    best_model = create_model(opt)
    # 加载具有最高验证精度的模型
    best_model.load_weights('best_model_'+opt+'.h5')
    best_model.compile(loss='mse', optimizer=opts[opt], metrics=['accuracy'])
    score = best_model.evaluate(X_test.values, y_test, verbose=0)
    results.append([opt, best_epoch, best_acc, score[1]])

Epoch 00201: early stopping
Epoch 00568: early stopping
Epoch 00482: early stopping
Epoch 00790: early stopping
Epoch 00543: early stopping
Epoch 00754: early stopping
Epoch 00517: early stopping
Epoch 01089: early stopping


In [13]:
# 比较结果
res = pd.DataFrame(results)
res.columns = ['optimizer', 'epochs', 'val_accuracy', 'test_accuracy']
print(res)

      optimizer  epochs  val_accuracy  test_accuracy
0           sgd       0      0.000000       0.000000
1      sgd-0001     367      0.584375       0.584375
2          adam     281      0.650000       0.650000
3      adadelta     589      0.634375       0.634375
4       rmsprop     342      0.637500       0.637500
5  rmsprop-0001     553      0.621875       0.621875
6         nadam     316      0.637500       0.637500
7        adamax     888      0.650000       0.650000


- 写于最后
    - 后面我会解释如何使用网格搜索来进行参数调优。
    - 网格搜索会寻找合适的优化器来调整神经网络学习过程。